## NYC Yellow Taxi 2019

### Data Loading

To view the root 'databricks-datasets' folder, run:
`display(dbutils.fs.ls("/databricks-datasets/"))`

In [0]:
display(dbutils.fs.ls("/databricks-datasets/nyctaxi/tripdata/yellow/"))

In [0]:
# Load in the data from August 2019
nyc_2019_08_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("dbfs:/databricks-datasets/nyctaxi/tripdata/yellow/yellow_tripdata_2019-08.csv.gz")

# Number of rows
nyc_2019_08_df.count()

In [0]:
# View schema
nyc_2019_08_df.printSchema()

### Data Cleaning and Aggregation

In [0]:
# Drop rows if missing pickup or dropoff datetime
nyc_2019_08_df = nyc_2019_08_df.na.drop(subset=["tpep_pickup_datetime", "tpep_dropoff_datetime"])
nyc_2019_08_df.count()
# Number of rows remains the same ==> no missing values

In [0]:
from pyspark.sql.functions import min, max, col

min_max_datetime = nyc_2019_08_df.select(min(col("tpep_pickup_datetime")), max(col("tpep_pickup_datetime")))
min_max_datetime.show()

In [0]:
# Filter rows to include only trips in August
nyc_2019_08_df_clean = nyc_2019_08_df.filter(
    (col("tpep_pickup_datetime") >= "2019-08-01") & (col("tpep_pickup_datetime") < "2019-09-01") & 
    (col("tpep_dropoff_datetime") >= "2019-08-01") & (col("tpep_dropoff_datetime") <= "2019-09-01")
)
nyc_2019_08_df_clean.count()

In [0]:
from pyspark.sql.functions import unix_timestamp

# Exclude trips with invalid durations: negative or exceeding 24 hours (86400 seconds)
nyc_2019_08_df_clean = nyc_2019_08_df_clean.filter(
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")).between(0, 86400)
)

nyc_2019_08_df_clean.count()

In [0]:
# Extract day and hour (military) from pickup and dropoff datetime
from pyspark.sql.functions import date_format, hour

for prefix, col_name in [("pickup", "tpep_pickup_datetime"), ("dropoff", "tpep_dropoff_datetime")]:
    nyc_2019_08_df_clean = nyc_2019_08_df_clean.withColumn(f"{prefix}_date", date_format(col_name, "yyyy-MM-dd"))\
                                   .withColumn(f"{prefix}_hour", hour(col_name))

nyc_2019_08_df_clean.count()

### Exploratory Data Analysis

In [0]:
# Which hour has the highest number of pickups?
nyc_2019_08_df_clean.createOrReplaceTempView("nyc_2019_08")

pickups_per_hours = spark.sql(
    """
    SELECT pickup_hour, count(*) as num_pickups FROM nyc_2019_08
    GROUP BY pickup_hour
    ORDER BY num_pickups DESC
    LIMIT 5
    """
)

pickups_per_hours.show()

In [0]:
# Which days had the highest number of pickups?
pickups_per_day = spark.sql(
    """
    SELECT pickup_date, count(*) AS num_pickups FROM nyc_2019_08
    GROUP BY pickup_date
    ORDER BY num_pickups DESC
    LIMIT 10
    """
)

pickups_per_day.show()